In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.app-tag-mapping.v1/dimension/_update_date=2020-07-03/_update_hour=13/

In [0]:


df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.app-tag-mapping.v1/dimension/")
#df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.app-tag-mapping.v1/dimension/")
df.show()


In [0]:

#from bdce.common.utils import update_application_code
# update_application_code(spark, 'BDP-DOM-UDS-DATAPIPELINE', 'app_rating_v4')
from pyspark.sql import types as T
from pyspark.sql import functions as F
from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view
#spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
#import aaplproxy
class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass
def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()

sql_text = """

--market_key: 0 - gp, 1 - ios, order alphabetically
WITH category_info AS (
  SELECT col1 AS id, col2 AS code, col3 AS type, col4 AS is_main_category, col5 AS legacy_category_id, col6 AS slug, col7 AS full_path, col8 AS legacy_offline_name
  FROM(
    VALUES(6714, VIDEO_PLAYERS, category, t, 23, video-players-editors, OVERALL > APPLICATION > VIDEO_PLAYERS, Video Players & Editors),
        (6715, MAPS_AND_NAVIGATION, category, t, 34, map-and-navigation, OVERALL > APPLICATION > , MAPS_AND_NAVIGATION, Maps & Navigation),
        (6716, bn_1722771031, category, f, 1722771031, education, Overall > Enseignement, , ))
);

WITH category_localized_info AS (
  SELECT col1 AS category_info_id, col2 AS localized_name, col3 AS language
  FROM(
    VALUES(6714, Другое ,ru),
          (6714, Casual, en),
          (6715, 休闲, zh-cn),
          (6716, Racing, en))
);

WITH unified_category_mapping AS (
  SELECT col1 AS unified_category_id, col2 AS category_id, col3 AS is_primary
  FROM(
    VALUES(800000, 400000, f),
          (800000, 100000, f),
          (800001, 100001, f),
          (800001, 400001, f),
          (800002, 100004, f),
          (800002, 100002, f),
          (800002, 400022, f),
          (800002, 400002, f))
);

WITH category_hierarchy_id AS (
  SELECT col1 AS id, col2 AS hierarchy_code, col3 AS country_code
  FROM(
    VALUES(49, , WW),
          (65, int-data, WW))
);

WITH category_hierarchy AS (
  SELECT col1 AS hierarchy_id, col2 AS category_id, col3 AS parent_category_id, col4 AS index
  FROM(
    VALUES(49, 800000, , 0),
          (49, 800001, , 1),
          (65, 800002, 800001, 0))
);

WITH category_feature AS(
  SELECT col1 A category_id, col2 AS feature_code, col3 AS support
  FROM(
    VALUES(800000, int-web, t),
          (800000, ss-top-chart, f),
          (800000, int-data, t),
          (800000, ss-detail, t),
          (800001, int-web, t),
          (800001, ss-top-chart, f),
          (800001, ss-detail, t),
          (800002, int-web, t),
          (800002, ss-top-chart, f))
);

WITH init_category AS (
  SELECT col1 AS category_id, col2 AS category_info_id, col3 AS market, col4 AS start_date
  FROM(
    VALUES(800000, 6714, all, 2000-01-01),
          (800001, 6715, all, 2000-01-01),
          (800002, 6716, all, 2000-01-01))
);
WITH init_feature AS(
  SELECT
    category_id,
    collect_set(feature_code) as feature_code_list
  FROM
    category_feature
  WHERE support = 't'
  GROUP BY category_id
);
--!DEBUG:SHOW SELECT * FROM init_feature;

WITH unified_category AS(
  SELECT
    unified_category_id as unified_category_key,
    is_primary
  FROM
    unified_category_mapping
  FULL JOIN init_category
  ON unified_category_mapping.category_id = init_category.category_id
);
--!DEBUG:SHOW SELECT * FROM unified_category;


"""
# store_unified , rank_unified
namespace = "aa.category.test.v2"
ingest_msg = {
  "namespace": "aa.store.test.v2",
  "job_type": "routine",
  "options":{

  },
  "source": [
    {
      "path": []
    }
  ]
}
run(spark, ingest_msg, sql_text)



In [0]:
%%sh
PGPASSWORD='8eRRt!g5^Sf6' psql -h 10.2.8.11 -p 5433 -Uapp_bdp_pipeline -d category_service << EOF
\d
EOF

In [0]:


from snowflake import connector
conn = connector.connect(
    autocommit=False,
    user='fzhu',
    password='Lily870104',
    account='appannie_aa_int_prod.us-east-1',
    role='DEV_PG',
    warehouse='WH_DOD_READ',
    database='AA_INTELLIGENCE_PRODUCTION',
    schema='adl_store_paid',
    session_parameters={
        'OPERATION_TAG': "test",
    }
)
cursor = conn.cursor()



